Nesse exercício, utilizaremos o conceito de indíce invertido, com a quantidade de ocorrências da palavra no documento, para a melhora na eficiência de consultas. Essa forma de armazenamento dos tokens, se dá mapeando documentos a palavras, ao invés do palavras a documentos, como vinhamos fazendo nos exercícios anteriores.

In [29]:
import pandas as pd
import collections
import heapq
import time
import operator
import csv
import nltk
import string
nltk.download('punkt')
nltk.download('rslp')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Questão 1


Construiremos abaixo o indíce invertido a partir da mesma coleção utilizada previamente, e o armazenaremos em um arquivo **.csv**.

In [30]:
data = pd.read_csv('./results.csv', sep=',')
data.head()

,title,subtitle,author,date,section,text,url
0,“A sociedade foi Rubens Paiva não os facínora...,A decisão da juíza que proíbe as Forças Armada...,F. M.,30/03/2019 00:11:08,Brasil,A juíza federal Ivani Silva da Luz de Brasíli...,https://brasil.elpais.com/brasil/2019/03/26/po...
1,Justiça suspende decisão que proibia Forças Ar...,Liminar havia sido concedida na sexta-feira a ...,Marina Rossi,30/03/2019 16:17:59,Brasil,Menos de 24 horas depois de a juíza federal Iv...,https://brasil.elpais.com/brasil/2019/03/30/po...
2,Governo Bolsonaro prega “negacionismo históric...,Marcos Napolitano professor da USP diz que o...,Regiane Oliveira,04/04/2019 22:37:48,Brasil,Quando determinou que de 31 de março 1964 u...,https://brasil.elpais.com/brasil/2019/04/05/po...
3,Quando os pais de Gabo perceberam que tinham u...,Gustavo Tatis percorre o universo de García Má...,Jesús Ruiz Mantilla,07/03/2019 16:38:56,Cultura,Quando era pequeno Luisa e Gabriel se preo...,https://brasil.elpais.com/brasil/2019/03/06/cu...
4,Rádios canadenses banem músicas de Michael Jac...,Quebec Cogeco Media toma a decisão após queixa...,Jaime Porras Ferreyra,07/03/2019 16:12:37,Cultura,Desde a manhã da última segunda-feira e ...,https://brasil.elpais.com/brasil/2019/03/06/cu...


In [31]:
textData = pd.DataFrame(data['text'], columns=['text'])
textData['tokenizedText'] = data.apply(lambda row: nltk.word_tokenize(row['text'].lower(), language='portuguese'), axis=1)
textData.head()

,text,tokenizedText
0,A juíza federal Ivani Silva da Luz de Brasíli...,"[a, juíza, federal, ivani, silva, da, luz, de,..."
1,Menos de 24 horas depois de a juíza federal Iv...,"[menos, de, 24, horas, depois, de, a, juíza, f..."
2,Quando determinou que de 31 de março 1964 u...,"[quando, determinou, que, de, 31, de, março, 1..."
3,Quando era pequeno Luisa e Gabriel se preo...,"[quando, era, pequeno, luisa, e, gabriel, se, ..."
4,Desde a manhã da última segunda-feira e ...,"[desde, a, manhã, da, última, segunda-feira, e..."


Acima, mostramos os documentos com os textos originais e os textos tokenizados. Agora iremos construir de fato o indíce invertido, removendo as **stopwords** e as palavras com tamanho menor que 3. 

Para a construção desse indíce, utilizaremos o *dicionário* de Python.

In [0]:
stopwords = nltk.corpus.stopwords.words('portuguese')

index = {}
document = 0

for wordList in textData.tokenizedText:
  document += 1
  for word in wordList:
    if word not in stopwords and len(word) >= 3:      
      if word not in index.keys():
        index[word] = []
      index[word].append(document)
      
for elem in index.items():
  d = dict(collections.Counter(elem[1]))
  index[elem[0]] = list(d.items())
  
        

Agora transformaremos esse indíce em um _dataframe_ da biblioteca _pandas_ e então o transformaremos em um arquivo **.csv**.

In [33]:
indexed_data = pd.DataFrame()

indexed_data['Word'] = [word for word in index.keys()]
indexed_data['Documents'] = [docs for docs in index.values()]

indexed_data.to_csv('indexed_data.csv', encoding='utf-8', index=False)

indexed_data.head(10)

,Word,Documents
0,juíza,"[(1, 2), (2, 1)]"
1,federal,"[(1, 2), (2, 2), (3, 1), (6, 1), (7, 3), (15, ..."
2,ivani,"[(1, 1), (2, 1)]"
3,silva,"[(1, 3), (2, 1), (6, 1), (14, 1), (26, 1), (73..."
4,luz,"[(1, 3), (2, 1), (9, 1), (17, 1), (32, 2), (78..."
5,brasília,"[(1, 1), (8, 1), (33, 1), (35, 1), (44, 1), (4..."
6,proibiu,"[(1, 1), (2, 1), (119, 1), (162, 1)]"
7,caráter,"[(1, 1), (15, 1), (36, 1), (60, 1), (89, 1), (..."
8,liminar,"[(1, 1), (2, 3), (119, 1), (217, 1)]"
9,nesta,"[(1, 2), (4, 1), (8, 1), (21, 1), (22, 2), (23..."


# Questão 2

Nessa parte do exercício, iremos implementar formas de consulta ao indíce que construímos anteriormente. 

Essas consultas são:

*   Documento-por-vez
*   Termo-por-vez

In [0]:
def documentAtATimeRetrieval(query, inverted_index, k):
    start = time.time()
    inverted_lists = []

    r = []
    for word in query.split(" "):
        if word in inverted_index.keys():
            inverted_lists.append(inverted_index[word])
    for document in range(1, len(data)+1):
        sd = 0
        for inverted_list in inverted_lists:
            for i in inverted_list:
                if (i[0] == document):
                    sd += i[1]
                    break
        if (sd != 0):
          heapq.heappush(r, (sd, document))
    
    end = time.time() - start
    return heapq.nlargest(k, r), end
  
def term_a_time(query, inverted_index, k):
    start = time.time()
    
    a = {}
    inverted_lists = []
    r = []
    for word in query.split(" "):
        if word in inverted_index.keys():
            inverted_lists.append(inverted_index[word])
    for inverted_list in inverted_lists:
        for post in inverted_list:
            d = post[0]
            freq = post[1]
            if (d in a.keys()):
                a[d] += freq
            else:
                a[d] = freq
    for (d, ad) in a.items():
        sd = ad
        heapq.heappush(r, (sd, d))
        
    end = time.time() - start
    return heapq.nlargest(k, r), end

Abaixo iremos criar um *dataFrame* para armazenar as consultas que realizaremos para testar os algoritmos.

In [38]:
queries = ["lula", "bolsonaro", "brasil", "economia", "ditadura"]
k = 5
results_doc = []
results_term = []
times_doc = []
times_term = []

for query in queries:
  score_doc, time_doc = documentAtATimeRetrieval(query, index, k)
  results_doc.append(score_doc)
  times_doc.append(time_doc)
  
  score_term, time_term = termAtATimeRetrieval(query, index, k)
  results_term.append(score_term)
  times_term.append(time_term)
  
queries_df = pd.DataFrame()
queries_df['Consulta'] = queries
queries_df['Documento por vez'] = results_doc
queries_df['Termo por vez'] = results_term
queries_df['Comparativo'] = queries_df['Documento por vez'] == queries_df['Documento por vez']
queries_df.index+=1
queries_df

,Consulta,Documento por vez,Termo por vez,Comparativo
1,lula,"[(9, 15), (3, 234), (3, 216), (2, 226), (2, 204)]","[(9, 15), (3, 234), (3, 216), (2, 226), (2, 204)]",True
2,bolsonaro,"[(42, 151), (34, 207), (32, 166), (22, 19), (1...","[(42, 151), (34, 207), (32, 166), (22, 19), (1...",True
3,brasil,"[(45, 151), (15, 166), (11, 19), (10, 26), (9,...","[(45, 151), (15, 166), (11, 19), (10, 26), (9,...",True
4,economia,"[(10, 138), (8, 125), (6, 127), (6, 69), (6, 34)]","[(10, 138), (8, 125), (6, 127), (6, 69), (6, 34)]",True
5,ditadura,"[(16, 7), (13, 238), (11, 216), (9, 172), (9, ...","[(16, 7), (13, 238), (11, 216), (9, 172), (9, ...",True


Abaixo iremos criar um *dataFrame* para mostrar os tempos de execução de cada um dos algoritmos implementados.

In [36]:
queries_tempo_df = pd.DataFrame()
queries_tempo_df['Tempo Médio: Documento por vez (ms)'] = times_doc
queries_tempo_df['Tempo Médio: Termo por vez (ms)'] = times_term
queries_tempo_df.index+=1
queries_tempo_df

,Tempo Médio: Documento por vez (ms),Tempo Médio: Termo por vez (ms)
1,0.001046,0.000043
2,0.000457,0.000017
3,0.000671,0.000026
4,0.001418,0.000060
5,0.002126,0.000050
